# Парсинг продукции Korloy

Я решил в качестве проекта по парсингу выбрать задачу по сбору обозначений и конструктивных параметров (размеров) сверл для обработки конструкционной стали на глубину до 7 диаметров. Данную задачу я выбрал посколько являюсь руководителем технического отдела в Mitsubishi Materials, которая выпускает металлорежущий инструмент и занимается его продажами. В своей работе я обнаружил, что у моих сотрудников уходит очень много времени на то, чтобы подобрать аналоги конкурентных инструментов для клиентов, так как на сегодняшний день данный подбор осуществляется сотрудниками в ручном режиме при помощи сравнения применения серии инструментов, а также их размеров. Я бы хотел автоматизировать процесс подбора аналогов, чтобы освободить время своим сотрудникам для более важных и сложных технических задач, например написание технологий обработки автомобильных двигателей или турбинных лопаток для авиадвигателей, что также входит в наши повседневные задачи. В связи с этим я задался целью написать программы для парсинга всей продукции наших конкурентов.

В рамках данного проекта, я выбрал сайт электронного каталога одного из наших конкуртов Korloy. С этого сайта мы будем парсить только одну серию сверл с диаметрами более 3 мм (т.к. в нашей продукции применяются в основном такие сверла). В своей дальнейшей работе в компании Mitsubishi Materials я планирую выполнить парсинг всей конкурентной продукции.

In [1]:
from selenium import webdriver #импортируем "робота", который проходит по страницам в хроме
from selenium.webdriver.common.keys import Keys #импортируем Keys для ввода данных в поле "Поиск" и нажатия на кнопки

In [2]:
#запускаем webdriver, открывается пустая страница браузера Chrome
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\79629\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache
C:\Users\79629\AppData\Local\Temp\ipykernel_17400\2127724492.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
#импортируем UserAgent для браузера Chrome
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.14 (KHTML, like Gecko) Chrome/24.0.1292.0 Safari/537.14'

In [4]:
#импортируем библиотеки для парсинга
import requests
from bs4 import BeautifulSoup
from time import sleep
url = "https://catalog.korloy.com/app/main.do"

#проверяем доступ к странице
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
print(response)

<Response [200]>


In [5]:
#переходим на сайт
driver.get(url)

In [6]:
#создаем функцию для клика по кнопкам
def click (fld):
    fld = driver.find_element("xpath", fld)
    fld.click()

In [7]:
#переходим на страницу монолитного режущего инструмента
click('/html/body/div[3]/div[6]/div[3]/div/div/div/div/div/div/ul/li[7]/a/span')
sleep (1.5) #спим 1.5 с до следующего запроса

In [8]:
#переходим в раздел сверл
#click('/html/body/div[3]/div[6]/div[2]/div[2]/div/div[1]/div[2]/div/ul/li[5]/a/div[3]/p')
#click('/html/body/div[3]/div[6]/div[2]/div[2]/div/div[1]/div[2]/div/ul/li[6]/a/div[3]/p')
click('/html/body/div[3]/div[6]/div[2]/div[2]/div/div[1]/div[2]/div/ul/li[9]/a/div[3]/p')
sleep(1.5)

In [9]:
#выбираем интересующую нас серию сверл (Drills_MSDP(H)-7P - твердосплавные для обработки конструкционных сталей на глубину до 7D)
click('/html/body/div[3]/div[6]/div[2]/div[2]/div/div[1]/div[2]/div[1]/div/ul/li[9]/a/div[3]/p')
sleep (1.5)

In [10]:
#скролим страницу до положения, чтобы была видна кнопка настройки фильтра
driver.execute_script("window.scrollTo(0, 600)")

#жмём кнопку для настройки фильров поиска
click('/html/body/div[3]/div[6]/div[2]/div[2]/div/div[1]/div[1]/div[2]/ul/li[2]/a')
sleep (1.5)


#выбираем необходимый диапазон диаметров сверл (диаметры не менее 3 мм)
click ('/html/body/div[3]/div[6]/div[2]/div[2]/div/div[1]/div[1]/div[3]/div[1]/div/table/tbody/tr[1]/td[2]/div/a[3]')
sleep (1.5)
#выбираем знак >=

#кликаем на поле с вводом диаметра
fld = driver.find_element("xpath", '/html/body/div[3]/div[6]/div[2]/div[2]/div/div[1]/div[1]/div[3]/div[1]/div/table/tbody/tr[1]/td[2]/div/div[1]/input')
sleep (1.5)
click('/html/body/div[3]/div[6]/div[2]/div[2]/div/div[1]/div[1]/div[3]/div[1]/div/table/tbody/tr[1]/td[2]/div/div[1]/input')
sleep (1.5)

#вводим диаметр 3 мм
diam = 3
fld.send_keys(diam) #ввод числа 3 в поле
sleep (1.5)

#жмём кнопку 'Search'
click('/html/body/div[3]/div[6]/div[2]/div[2]/div/div[1]/div[1]/div[3]/div[2]/div/a[2]')
sleep (1.5)

#обнаруживаем, что фильтр на сайте не работает (возможно, пофиксят в будущем). Для того, чтобы убедиться в том, что фильтры
#на сайте не работают, я также сделал это вручную
#делаем вывод о том, что фильтрацию данных придется делать после парсинга таблицы целиком

In [11]:
#импортируем tqdm для отслеживания процесса парсинга
from tqdm import tqdm

In [12]:
#собираем разметку страницы и "варим суп"
for i in tqdm(range(100)):
    page = driver.page_source
    sleep (0.5)
soup = BeautifulSoup(page, 'lxml')

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:02<00:00,  1.60it/s]


In [13]:
#изучив дерево сайта, делаем вывод, что обозначения сверл и их конструктивные параметры (значения размеров на эскизе) хранятся
#в 2-х разных таблицах, чего можно не заметить визуально на странице сайта. Для начала соберем обозначения сверл из первой
#таблицы и запишем их в список

designation = []
table = soup.find('table', {'id' : 'table_itemList'}) #находим таблицу с обозначением сверл

In [14]:
#сохраняем обозначения в список
for tr in table.find_all('tr'):      
    for td in tr.find_all('td', {'class' : 'ellipsis'}):
        designation.append(td.text)    

In [15]:
Korloy_tools = {'Designation' : designation}

In [16]:
table_sizes = table.find('tbody') #находим таблицу с размерами сверл

In [17]:
#собираем названия размеров из шапки таблицы
sizes_names = []
ths = table_sizes.find_all('th')
for th in ths:
    sizes_names.append(th.text)

In [18]:
#добавляем обозначения размеров в качестве ключей словаря Korloy_tools. Значениями ключей будут пустые списки
for size_name in sizes_names:
    Korloy_tools[size_name] = []

In [19]:
#заполняем словарь, путем добавления в списки размеров сверл
trs = table_sizes.find_all('tr')

i = 0
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        Korloy_tools[sizes_names[i]].append(td.text)
        i += 1
        if i == len(sizes_names):
            i = 0

In [20]:
#импортируем pandas для работы с DataFrame
import pandas as pd

In [21]:
#создаем DataFrame из словаря Korloy_tools
df = pd.DataFrame(Korloy_tools)

In [22]:
#удаляем строки с позициями, для которых не указаны диаметры DC
df = df.loc[df['DC'] != '']

In [23]:
#преобразуем числа в DataFrame в тип данных float
for key in Korloy_tools:
    if key != 'Designation':
        df[key] = df[key].astype('float')

In [24]:
#удаляем строки с позициями, для которых диаметры DC < 3
df = df.loc[df['DC'] - 3 >= 0]

In [25]:
#закрываем Webdriver
driver.close()

In [26]:
#Сохраняем DataFrame в файл Excel
df.to_excel('parsed_drills.xlsx', index=False)
df.to_pickle('parsed_drills.pkl')

In [27]:
import sqlite3

In [28]:
conn = sqlite3.connect('Korloy.db')
cursor = conn.cursor()

In [29]:
try:
    query = "CREATE TABLE \"Drills\" (\"Designation\" TEXT, \"DC\" FLOAT, \"DCON-MS\" FLOAT, \"LU\" FLOAT, \"LCF\" FLOAT, \"LF\" FLOAT, \"OAL\" FLOAT, \"SIG\" FLOAT, \"WT\" FLOAT, \"ULDR\" FLOAT, \"NOF\" FLOAT )"
    cursor.execute(query)
    
except:
    pass

In [31]:
df.to_sql('Drills', conn, if_exists='replace', index = False)

342